In [ ]:
### Import libraries
from netCDF4 import Dataset
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
ti = 0
lev = 600

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
full = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')
large = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_large/wrfout_d01_2018-11-13_12_00_00')
small = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_small/wrfout_d01_2018-11-13_12_00_00')
box = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_box/wrfout_d01_2018-11-13_12_00_00')
hole = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_hole/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_full = np.asarray(getvar(full,'pressure',timeidx=ti))
p_large = np.asarray(getvar(large,'pressure',timeidx=ti))
p_small = np.asarray(getvar(small,'pressure',timeidx=ti))
p_box = np.asarray(getvar(box,'pressure',timeidx=ti))
p_hole = np.asarray(getvar(hole,'pressure',timeidx=ti))

### Height
z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
zl_ctl = np.asarray(interplevel(z_ctl, p_ctl, lev))

### Theta
tl_ctl = np.asarray(getvar(ctl,'THM',timeidx=ti))[23,:,:]
dtl_full = np.asarray(getvar(full,'THM',timeidx=ti))[23,:,:] - tl_ctl
dtl_large = np.asarray(getvar(large,'THM',timeidx=ti))[23,:,:] - tl_ctl
dtl_small = np.asarray(getvar(small,'THM',timeidx=ti))[23,:,:] - tl_ctl
dtl_box = np.asarray(getvar(box,'THM',timeidx=ti))[23,:,:] - tl_ctl
dtl_hole = np.asarray(getvar(hole,'THM',timeidx=ti))[23,:,:] - tl_ctl


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.25

### Plot subdomain
min_lat = 15
max_lat = 60
min_lon = -125
max_lon = -75

buf = 25
zl_ctl[(lons < min_lon-buf) | (lons > max_lon+buf)] = np.NaN

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],
                               ['bottomleft','bottommiddle','bottomright'],
                               ['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(6.8,4.6), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels=False

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('Control')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)
axd['topleft'].text(0.12, 0.55, 'No Perturbations',transform=axd['topleft'].transAxes + trans, fontsize=12,verticalalignment='top', bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
##############

# Add the gridlines
gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topmiddle'].contourf(lons, lats, dtl_box, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

min_lat1 = 25
max_lat1 = 40
min_lon1 = -107
max_lon1 = -92

axd['topmiddle'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='g', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topmiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('Target')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
##############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topright'].contourf(lons, lats, dtl_small, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('Short $\lambda$')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
##############

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.rotate_labels=False

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomleft'].contourf(lons, lats, dtl_full, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('Full')
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
##############

# Add the gridlines
gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.rotate_labels=False
gls.left_labels=False


cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottommiddle'].contourf(lons, lats, dtl_hole, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

axd['bottommiddle'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='g', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottommiddle'].set_title('Target$^\complement$')
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
##############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False
gls.rotate_labels=False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomright'].contourf(lons, lats, dtl_large, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('Long $\lambda$')
axd['bottomright'].text(0.0, 1.0, '(f)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_png/30km_exp_setup.png',bbox_inches='tight')
plt.show()
